In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from func.ml_utils import get_factorize_feature
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
logger = logger_func()

2019-10-02 01:50:23,212 func.utils 347 [INFO]    [logger_func] start 


In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMN_GROUP = 'DT-M'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, COLUMN_GROUP, 'is_train', 'date']

def filter_feature(path):
    if path.count(''):
        return True
    else:
        return False

# paths_train = glob('../submit/re_sub/50*_train.gz')
# paths_test  = glob('../submit/re_sub/50*_test.gz')
# paths_train += glob('../submit/re_sub/Tran*_train.gz')
# paths_test  += glob('../submit/re_sub/Tran*_test.gz')
# paths_train += glob('../submit/re_sub/is*_train.gz')
# paths_test  += glob('../submit/re_sub/is*_test.gz')

paths_train = glob('../submit/re_sub/*_train.gz')
paths_test  = glob('../submit/re_sub/*_test.gz')
# paths_train += glob('../submit/add_feature/*_train.gz')
# paths_test  += glob('../submit/add_feature/*_test.gz')

print(len(paths_train))
# sys.exit()
# paths_train += glob('../feature/valid_trush/528*uid2*_train.gz')
# paths_test  += glob('../feature/valid_trush/528*uid2*_test.gz')

# for path in paths_train:
#     if path.count('C14_ratio'):
#         paths_train.remove(path)
        
# for path in paths_test:
#     if path.count('C14_ratio'):
#         paths_test.remove(path)
    
# paths_train = glob('../feature/raw_use/*_train.gz')
# paths_test = glob('../feature/raw_use/*_test.gz')
# paths_train = [path for path in paths_train if filter_feature(path) ]
# paths_test = [path for path in paths_test if filter_feature(path) ]

# paths_train_feature = sorted(glob('../feature/org_use/*_train.gz'))
# paths_test_feature  = sorted(glob('../feature/org_use/*_test.gz'))

# paths_train_feature += sorted(glob('../feature/valid/*_train.gz'))
# paths_test_feature  += sorted(glob('../feature/valid/*_test.gz'))

# paths_train_feature += sorted(glob('../feature/kernel/*_train.gz'))
# paths_test_feature  += sorted(glob('../feature/kernel/*_test.gz'))

# paths_train_feature = sorted(glob('../feature/valid_use/*_train.gz'))
# paths_test_feature  = sorted(glob('../feature/valid_use/*_test.gz'))
paths_train_feature = []
paths_test_feature  = []

# df_train = reduce_mem_usage( parallel_load_data(paths_train) )
# df_test  = reduce_mem_usage( parallel_load_data(paths_test) )
df_train = parallel_load_data(paths_train)
df_test  = parallel_load_data(paths_test)
Y = df_train[COLUMN_TARGET]
df_train.drop(COLUMN_TARGET, axis=1, inplace=True)

1807


Process ForkPoolWorker-59:
Process ForkPoolWorker-49:
Process ForkPoolWorker-48:
Process ForkPoolWorker-91:
Process ForkPoolWorker-87:
Process ForkPoolWorker-90:
Process ForkPoolWorker-89:
Process ForkPoolWorker-68:
Process ForkPoolWorker-83:
Process ForkPoolWorker-84:
Process ForkPoolWorker-88:
Process ForkPoolWorker-94:
Process ForkPoolWorker-79:
Process ForkPoolWorker-85:
Process ForkPoolWorker-93:
Process ForkPoolWorker-96:
Process ForkPoolWorker-30:
Process ForkPoolWorker-4:
Process ForkPoolWorker-60:
Process ForkPoolWorker-57:
Process ForkPoolWorker-63:
Process ForkPoolWorker-28:
Process ForkPoolWorker-38:
Process ForkPoolWorker-46:
Process ForkPoolWorker-58:
Process ForkPoolWorker-70:
Process ForkPoolWorker-18:
Process ForkPoolWorker-53:
Process ForkPoolWorker-19:
Process ForkPoolWorker-21:
Process ForkPoolWorker-42:
Process ForkPoolWorker-41:
Process ForkPoolWorker-24:
Process ForkPoolWorker-16:
Process ForkPoolWorker-26:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Proc

In [3]:
data = pd.concat([df_train, df_test], axis=0)

cols_categorical = [col for col in df_train.columns 
                    if not col in COLUMNS_IGNORE
                    and 
                    (
                    col.count('uid')
                    or col.count('ugr')
                    or col.count('60')
                    )
                   ]

list_drop = []
for col in tqdm(cols_categorical):
    cnt = df_train[col].head(2000).value_counts().shape[0]
    if cnt > 100:
        list_drop.append(col)
        continue
    cnt = data[col].value_counts().shape[0]
    if cnt > 200:
        list_drop.append(col)
    else:
        data[col].fillna(-99999, inplace=True)
            
cols_categorical = list(set(cols_categorical) -set(list_drop))
print(len(cols_categorical))
data = get_factorize_feature(data, cols_categorical)
df_train = data.iloc[:len(df_train)]
df_test  = data.iloc[len(df_train):]

# same_user_path = '../output/same_user_pattern/20190901_user_ids_share.csv'
# same_user_path = '../output/same_user_pattern/20190901_user_ids_share.csv'
# bear = pd.read_csv(same_user_path)
# bear = bear[[COLUMN_ID, 'predicted_user_id']]
# max_id = bear['predicted_user_id'].max()
# bear.loc[bear[bear['predicted_user_id'].isnull()].index, 'predicted_user_id'] = np.arange(
#     bear['predicted_user_id'].isnull().sum() ) + 1 + max_id
# bear['predicted_user_id'] =  bear['predicted_user_id'].astype('int')
# bear.shape

100%|██████████| 1017/1017 [00:08<00:00, 115.50it/s]


52


In [4]:
is_submit = [True, False][0]
n_splits = 6
set_type = 'new_set'

valid_paths_train = paths_train_feature[:]
valid_paths_test  = paths_test_feature[:]

#========================================================================
# pathの存在チェック。なぜかたびたびFileNotFoundErrorが起きるので,,,
#========================================================================
remove_paths = []
for trn_path, tes_path in zip(valid_paths_train, valid_paths_test):
    if os.path.exists(trn_path) and os.path.exists(tes_path):
        pass
    else:
        remove_paths.append(trn_path)
        remove_paths.append(tes_path)
for path in remove_paths:
    if path.count('train'):
        valid_paths_train.remove(path)
        print(f'remove {path}')
    elif path.count('test'):
        valid_paths_test.remove(path)
        print(f'remove {path}')

if len(valid_paths_train):
    df_feat_train = parallel_load_data(valid_paths_train)
    df_feat_test  = parallel_load_data(valid_paths_test)
    
    col_drops = eval_check_feature(df_feat_train, df_feat_test)
    
    tmp_train = df_train.join(df_feat_train)
    tmp_test = df_test.join(df_feat_test)
else:
    tmp_train = df_train
    tmp_test = df_test

#========================================================================
# Train Test で片方に存在しないFeatureを除外
#========================================================================
diff_cols = list(set(tmp_train.columns) - set(tmp_test.columns))

for col in list(set(diff_cols)):
    from_dir = 'valid'
    to_dir = 'valid_trush'
    move_feature([col], from_dir, to_dir)
tmp_train.drop(diff_cols, axis=1, inplace=True)
print(f"  * Diff Features: {len(diff_cols)}")

# same_user_path = '../output/same_user_pattern/0902__same_user_id__card_addr_pemail_M.csv'
# tmp_train = tmp_train.merge(bear[[COLUMN_ID, 'predicted_user_id']], how='inner', on=COLUMN_ID)
# COLUMN_GROUP = 'predicted_user_id'
# COLUMNS_IGNORE.append('predicted_user_id')

### DT-M
group_kfold_path = '../input/0908_ieee__DT-M_GroupKFold.gz'
group = read_pkl_gzip(group_kfold_path)
tmp_train[COLUMN_GROUP] = group

# tmp_train[COLUMN_GROUP] = tmp_train['528__ugr_uid3_Regist_date_agg_V95_137_mean_mean'].fillna(0)

#========================================================================
# Features elimination 
#==============================================================
# from scipy.stats import ks_2samp
# features_check = []
# columns_to_check = set(list(tmp_train)).difference(COLUMNS_IGNORE)
# for i in columns_to_check:
#     features_check.append(ks_2samp(tmp_test[i], tmp_train[i])[1])

# features_check = pd.Series(features_check, index=columns_to_check).sort_values() 
# features_discard = list(features_check[features_check==0].index)
# print(features_discard)
# tmp_train.drop(features_discard, axis=1, inplace=True)
# tmp_test.drop(features_discard, axis=1, inplace=True)


model_type = "cat"

  * Diff Features: 0


In [ ]:
params = {
#     'n_jobs': 96,
    'seed': 1208,
#     'subsample': 0.9,
    'num_boost_round':3000,
    'random_seed': 1208,
    'has_time': True,
    'max_depth': 10,
    'loss_function': 'Logloss',
    'custom_loss':['AUC'],
#     'logging_level':'Silent',
    'task_type' : 'CPU',
    'early_stopping_rounds' : 100,
    'n_splits': n_splits,
    'model_type': model_type,
    'fold': ['stratified', 'group'][1],
}

if is_submit:
    params['learning_rate'] = 0.01
#     params['learning_rate'] = 0.05
#     params['learning_rate'] = 0.1
    params["early_stopping_rounds"] = 100

use_cols = [col for col in df_train.columns if col not in COLUMNS_IGNORE]
logger.info(f"* EXP: dataset {set_type} {tmp_train.shape} lr {params['learning_rate']} ")
            
if False:
#     sample = tmp_train.sample(10000)
#     sample[COLUMN_GROUP].value_counts()
    use_cols = [col for col in df_train.columns if col not in COLUMNS_IGNORE][:10]
    params['early_stopping_rounds'] = 1
    params['num_boost_round'] = 1

feim, _ = eval_train(
    logger,
    tmp_train,
    Y,
    tmp_test,
#     sample,
#     Y.head(10000),
#     tmp_test.head(1000),
    COLUMN_GROUP,
    model_type,
    params,
    is_adv=[True, False][1],
    is_viz=[True, False][1],
    cols_categorical=cols_categorical
)
feim = list_result_feim[0]

2019-10-02 02:06:10,072 func.utils 26 [INFO]    [<module>] * EXP: dataset new_set (590540, 1808) lr 0.01  
INFO:func.utils:* EXP: dataset new_set (590540, 1808) lr 0.01 


0:	learn: 0.6752014	total: 3.68s	remaining: 3h 3m 49s
1:	learn: 0.6582513	total: 6.79s	remaining: 2h 49m 31s
2:	learn: 0.6415703	total: 9.64s	remaining: 2h 40m 31s
3:	learn: 0.6256069	total: 12.4s	remaining: 2h 35m 13s
4:	learn: 0.6095478	total: 15.2s	remaining: 2h 31m 34s
5:	learn: 0.5943644	total: 17.8s	remaining: 2h 28m 4s
6:	learn: 0.5788089	total: 20.6s	remaining: 2h 26m 46s
7:	learn: 0.5648888	total: 23.3s	remaining: 2h 25m 21s
8:	learn: 0.5507965	total: 26.2s	remaining: 2h 24m 58s
9:	learn: 0.5375087	total: 28.8s	remaining: 2h 23m 33s
10:	learn: 0.5240598	total: 31.7s	remaining: 2h 23m 24s
11:	learn: 0.5114841	total: 34.3s	remaining: 2h 22m 26s
12:	learn: 0.4997346	total: 36.9s	remaining: 2h 21m 25s
13:	learn: 0.4871548	total: 39.5s	remaining: 2h 20m 26s
14:	learn: 0.4761428	total: 42.1s	remaining: 2h 19m 40s
15:	learn: 0.4650386	total: 44.8s	remaining: 2h 19m 18s
16:	learn: 0.4541351	total: 47.6s	remaining: 2h 19m 19s
17:	learn: 0.4438933	total: 50.2s	remaining: 2h 18m 43s
18:	

In [17]:
11

11